In [1]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# importing necessary libraries
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from sklearn import preprocessing, model_selection


from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle

from pyparsing import Word, hexnums, WordEnd, Optional, alphas, alphanums

In [ ]:
class FeatureExtraction:
    def __init__(self, trainLabelsCSVPath):
        self.classes = pd.read_csv(trainLabelsCSVPath)

  #returns a data fram containing the frequency of the hexadecimal in the file
    def feature_extraction_bytes(self, folder_path):
        total_dict = []
        #loops through a list of all files in the folder
        for file in os.listdir(folder_path):
          #checks if the file has a byte extension
          #processes only byte files
            if file.find('bytes') != -1:
                # print('current file', file)
                file_path = os.path.join(folder_path, file)
                # get unique count
                uniq = {}
                #updates the dictionary uniq with the hexcode as keys and their frequency as values
                #reads the byte file
                with open(file_path) as f:
                    lines = f.read().splitlines()
                    f.close()
                    for line in lines:
                      #split each line by space
                      #the first item is ignored since it is not a hexadecimal
                        byt = line.split(' ')[1:]
                        #looping through the hexadecimals
                        for hexa in byt:
                            if len(hexa) > 1:
                                if hexa in uniq:
                                    uniq[hexa] += 1
                                else:
                                    uniq[hexa] = 1

                # get the class value 
                label = self.classes.loc[self.classes['Id'] == file.split('.')[0]]['Class'].values[0]
                uniq['Class'] = label
                # print(len(uniq))
                # print(uniq)
                total_dict.append(uniq)

                data = pd.DataFrame(total_dict)
        return data

    # extract opcode

    def get_opcode_count(self, file_path):
        opcode_dic = {}

        hex_integer = Word(hexnums) + WordEnd()  # use WordEnd to avoid parsing leading a-f of non-hex numbers as a hex
        line = ".text:" + hex_integer + Optional(
            (hex_integer * (1,))("instructions") + Word(alphas, alphanums)("opcode"))

        f = open(file_path, 'r', encoding='latin-1')
        source = f.read().splitlines()
        f.close()

        for source_line in source:
            try:
                if '.text:' in source_line:
                    result = line.parseString(source_line)
                    if "opcode" in result:
                        if result.opcode in opcode_dic:
                            opcode_dic[result.opcode] += 1
                        else:
                            opcode_dic[result.opcode] = 1
                    # print(result.opcode, result.instructions.asList())
            except:
                continue
        return opcode_dic

    # get section count
    def get_section_count(self, file_path):
        section_dic = {}

        line = "." + Word(alphas, alphanums)("section")

        f = open(file_path, 'r', encoding='latin-1')
        source = f.read().splitlines()
        f.close()

        for source_line in source:
            try:
                result = line.parseString(source_line)
                if "section" in result:
                    if result.section in section_dic:
                        section_dic[result.section] += 1
                    else:
                        section_dic[result.section] = 1
                    # print('section', result.section)
            except:
                continue
        return section_dic

    def feature_extraction_opcode(self, folder_path):
        total_dict = []
        for file in os.listdir(folder_path):
            if file.find('asm') != -1:
                # print('current file', file)
                # get unique count
                file_path = os.path.join(folder_path, file)
                uniq = self.get_opcode_count(file_path)

                # get class value
                label = self.classes.loc[self.classes['Id'] == file.split('.')[0]]['Class'].values[0]
                uniq['Class'] = label
                # print(len(uniq))
                # print(uniq)
                total_dict.append(uniq)

                data = pd.DataFrame(total_dict)
        return data

    def feature_extraction_section(self, folder_path):
        total_dict = []
        for file in os.listdir(folder_path):
            if file.find('asm') != -1:
                # print('current file', file)
                # get unique count
                file_path = os.path.join(folder_path, file)
                uniq = self.get_section_count(file_path)

                # get class value
                label = self.classes.loc[self.classes['Id'] == file.split('.')[0]]['Class'].values[0]
                uniq['Class'] = label
                # print(len(uniq))
                # print(uniq)
                total_dict.append(uniq)

                data = pd.DataFrame(total_dict)
        return data

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#import the necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt

class Model:
    def __init_subclass__(cls) -> None:
        pass

    def decisionTree(self, X_train, y_train, X_test, y_test):
        # training a DescisionTreeClassifier
        print('DescisionTreeClassifier')

        # dtree_model = DecisionTreeClassifier(criterion="entropy", max_depth=3).fit(X_train, y_train)
        dtree_model = DecisionTreeClassifier(criterion="gini", max_depth=3).fit(X_train, y_train)
        dtree_predictions = dtree_model.predict(X_test)

        # Metrics evaluation
        accuracy = metrics.accuracy_score(y_test, dtree_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, dtree_predictions,
                                                                               average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("tpr = recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, dtree_predictions)
        return accuracy, precision, recall, fscore, fpr

    # training a linear SVM classifier
    def SVClassifier(self, X_train, y_train, X_test, y_test):
        print('SVC Classifier')
        svm_model_linear = SVC(kernel='linear', C=1).fit(X_train, y_train)
        svm_predictions = svm_model_linear.predict(X_test)

        # creating a confusion matrix
        # cm = confusion_matrix(y_test, svm_predictions)

        # Metrics evaluation
        accuracy = metrics.accuracy_score(y_test, svm_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, svm_predictions,
                                                                               average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, svm_predictions)
        return accuracy, precision, recall, fscore, fpr

    # training a KNN classifier
    def KNearestClassifier(self, X_train, y_train, X_test, y_test):
        print('KNearestClassifier')
        knn = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)

        # prediction matrix
        knn_predictions = knn.predict(X_test)

        # Metrics evaluation
        accuracy = metrics.accuracy_score(y_test, knn_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, knn_predictions,
                                                                               average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, knn_predictions)
        return accuracy, precision, recall, fscore, fpr

    # training a Naive Bayes classifier
    def NaiveBayes(self, X_train, y_train, X_test, y_test):
        print('NaiveBayes')
        gnb = GaussianNB().fit(X_train, y_train)
        gnb_predictions = gnb.predict(X_test)

        # accuracy on X_test
        accuracy = gnb.score(X_test, y_test)
        print(accuracy)

        # creating a confusion matrix
        cm = confusion_matrix(y_test, gnb_predictions)

        # Metrics evaluation
        accuracy = metrics.accuracy_score(y_test, gnb_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, gnb_predictions,
                                                                               average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, gnb_predictions)
        return accuracy, precision, recall, fscore, fpr

    def SGD(self, X_train, y_train, X_test, y_test):
        print('SGD')
        clf = SGDClassifier(alpha=0.001, max_iter=100).fit(X_train, y_train)
        clf_predictions = clf.predict(X_test)

        # Model evaluation
        accuracy = metrics.accuracy_score(y_test, clf_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, clf_predictions,
                                                                               average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, clf_predictions)
        return accuracy, precision, recall, fscore, fpr

    def LogisticReg(self, X_train, y_train, X_test, y_test):
        print('Logistic Regression')
        clf = LogisticRegression(random_state=0).fit(X_train, y_train)
        clf_predictions = clf.predict(X_test)

        # Model evaluation
        accuracy = metrics.accuracy_score(y_test, clf_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, clf_predictions,
                                                                               average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, clf_predictions)
        return accuracy, precision, recall, fscore, fpr

    def RandomForest(self, X_train, y_train, X_test, y_test):
        print('RandomForest')
        clf = RandomForestClassifier(max_depth=2, random_state=0)
        clf.fit(X_train, y_train)
        clf_predictions = clf.predict(X_test)

        # Model evaluation
        accuracy = metrics.accuracy_score(y_test, clf_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, clf_predictions,
                                                                               average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, clf_predictions)
        return accuracy, precision, recall, fscore, fpr

    def DnnPreprocessing(self, X, Y):
        X = np.array(X)

        # Transform name species into numerical values
        encoder = LabelEncoder()
        encoder.fit(Y)
        Y = encoder.transform(Y)
        Y = np_utils.to_categorical(Y)

        # split the data into train and test
        train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1, random_state=0)
        return train_x, test_x, train_y, test_y, encoder

    def DNN_Model(self, train_x, test_x, train_y, test_y, epoch, encoder):
        model = Sequential()
        model.add(Dense(8, input_dim=train_x.shape[1], activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(train_y.shape[1], activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        history = model.fit(train_x, train_y, epochs=epoch, batch_size=2, validation_data=(test_x, test_y))

        scores = model.evaluate(test_x, test_y)
        print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

        predictions = model.predict(test_x)
        prediction_ = np.argmax(predictions, axis=1)
        prediction_ = encoder.inverse_transform(prediction_)

        test_y_ = np.argmax(test_y, axis=1)
        test_y_ = encoder.inverse_transform(test_y_)

        # Model evaluation
        accuracy = metrics.accuracy_score(test_y_, prediction_)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(test_y_, prediction_, average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        self.plot_Acc(history, epoch)
        fpr = self.calculate_tpr_fpr(test_y_, prediction_)
        return accuracy, precision, recall, fscore, fpr

    def plot_Acc(self, history, epoch):
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']

        loss = history.history['loss']
        val_loss = history.history['val_loss']

        epochs_range = range(epoch)

        plt.figure(figsize=(8, 8))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Training Accuracy')
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')
        plt.legend(loc='lower right')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')

        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Training Loss')
        plt.plot(epochs_range, val_loss, label='Validation Loss')
        plt.legend(loc='upper right')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()

    def calculate_tpr_fpr(self, true_y, pred_y):
        # creating a confusion matrix
        confusion_matrix = metrics.confusion_matrix(true_y, pred_y)

        FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
        FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
        TP = np.diag(confusion_matrix)
        TN = confusion_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        # Sensitivity, hit rate, recall, or true positive rate
        # TPR = TP / (TP + FN)
        # Fall out or false positive rate
        FPR = FP / (FP + TN)
        print('FPR', np.average(FPR))
        return np.average(FPR)